In [ ]:
bedtools_path = "/usr/bin/bedtools"
FASTA="Sus_scrofa.Sscrofa11.1.dna.chromosome.1.fa"

import pandas as pd
import numpy as np
from Bio import SeqIO
import random
import os

: 

In [3]:
df_del = pd.DataFrame(columns = [1,2,3,4])
df_dup = pd.DataFrame(columns = [1,2,3,4])
pathOut = "train/"
if pathOut != "" and pathOut.endswith("/") == False:
    pathOut += "/"
out = open(pathOut + "chrs.bed","w")

In [4]:
print("Generating duplication and deletion coordinates")
for fasta_file in SeqIO.parse(open(FASTA),"fasta"):
    #first line is fasta ID, 1(?), length of fasta file
    first_line = "\t".join([fasta_file.id,"1",str(len(str(fasta_file.seq)))]) + "\n"
    out.write(first_line)
    dup_lengths = []
    del_lengths = []
    #why 1000000 in cnv count? -> because we calculate it for megabase 
    max_cnv_length = 1e5
    cnv_count = int((len(str(fasta_file.seq))/max_cnv_length)/2)  #number of cnv that can fit in data devided by two because there are two types of cnv (duplications and deletions)
    while len(dup_lengths) < cnv_count:
        cnv_range = random.randint(50,100000)
        dup_lengths.append(cnv_range)
    while len(del_lengths) < cnv_count:
        cnv_range = random.randint(50,100000)
        del_lengths.append(cnv_range)
    # dodać tutaj test -> assert, że suma tych dwóch list będzie mniejsza od długości całego genomu
    dup_start = np.random.randint(1,len(str(fasta_file.seq)), size=(1, cnv_count))[0]
    del_start = np.random.randint(1,len(str(fasta_file.seq)), size=(1, cnv_count))[0]
    dup_ends = [int(a + b) for a, b in zip(dup_start, dup_lengths)]
    del_ends = [int(a + b) for a, b in zip(del_start, del_lengths)]
    dups = pd.DataFrame({1:[fasta_file.id]*cnv_count,2:dup_start,3:dup_ends,4:dup_lengths})
    dels = pd.DataFrame({1:[fasta_file.id]*cnv_count,2:del_start,3:del_ends,4:del_lengths})
    #tutaj zmienić ten append
    df_dup = df_dup.append(dups)
    df_del = df_del.append(dels)
out.close()
df_dup.to_csv(pathOut + "dup.bed",header=False,index=False,sep="\t")
df_del.to_csv(pathOut + "del.bed",header=False,index=False,sep="\t")
os.system(bedtools_path + " sort -i " + pathOut + "dup.bed | " + bedtools_path + " merge -i stdin > " + pathOut + "dup2.bed")
os.system(bedtools_path + " sort -i " + pathOut + "del.bed | " + bedtools_path + " merge -i stdin > " + pathOut + "del2.bed")
os.system("cp "+ pathOut + "del2.bed "+ pathOut + "del3.bed")
os.system("cp "+ pathOut + "dup2.bed "+ pathOut + "dup3.bed")
os.system(bedtools_path + " intersect -wa -v -a " + pathOut + "dup3.bed -b " + pathOut + "del3.bed > " + pathOut + "dup4.bed")
os.system(bedtools_path + " intersect -wa -v -a " + pathOut + "del3.bed -b " + pathOut + "dup3.bed > " + pathOut + "del4.bed")
no_chrs = list(range(1, int(1)+1))
chr_freq = {}
for i in no_chrs:
    chr_freq[i] = i/1
no_chrs = list(range(1, int(1)+1))
chr_freq = {}



Generating duplication and deletion coordinates
[[145916573 234139115  17681421 ... 234175821  81734871 239381829]]
[1]


/tmp/ipykernel_8674/190941863.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dup = df_dup.append(dups)
/tmp/ipykernel_8674/190941863.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_del = df_del.append(dels)


In [5]:
print("Generating duplication and deletion frequencies")
print(no_chrs)
for i in no_chrs:
    chr_freq[i] = round(i/1,3)
for i in ["del","dup"]:
    out = open(pathOut + str(i) + "5.bed","w")
    for line in open(pathOut + i + "4.bed"):
        if i == "del":
            num = random.randint(1,1)
            out.write(line.rstrip() + "\tdel\t" + str(chr_freq[num]) + "\t0\n")
        elif i == "dup":
            num = random.randint(1,1)
            count = np.random.choice([2,3,4,5,6,7,8,9,10], 1, p=[0.5, 0.1, 0.1, 0.05, 0.05,0.05,0.05,0.05,0.05])[0]
            freqs = num/1
            cp = (count*freqs) + ((1-freqs) * 1)
            while cp == 1.0:
                num = random.randint(1,1)
                count = np.random.choice([2,3,4,5,6,7,8,9,10], 1, p=[0.5, 0.1, 0.1, 0.05, 0.05,0.05,0.05,0.05,0.05])[0]
            out.write(line.rstrip() + "\tdup\t" + str(chr_freq[num]) + "\t" + str(count) + "\n")
    out.close()
    for j in chr_freq:
        out = open(pathOut + i + "." + str(j) + ".bed","w")
        for line in open(pathOut + i + "5.bed"):
            if float(line.split()[4]) >= chr_freq[j]:
                out.write(line)
        out.close()

Generating duplication and deletion frequencies
[1]


In [6]:
print("Removing overlaps, generating total file")
for i in no_chrs:
    print("Creating bedfiles for sample " + str(i))
    os.system("bedtools makewindows -b " + pathOut + "chrs.bed -w 5 > " + pathOut + "normal." + str(i) + ".bed")
    os.system(bedtools_path + " intersect -v -wa -a " + pathOut + "normal." + str(i) + ".bed -b " + pathOut + "dup." + str(i) + ".bed | " + bedtools_path + " intersect -v -wa -a stdin -b " + pathOut + "del." + str(i) + ".bed | " + bedtools_path + " sort -i stdin | " + bedtools_path + " merge -i stdin > " + pathOut + "normal2." + str(i) + ".bed")
    out = open(pathOut + "normal3." + str(i) + ".bed","w")
    for line in open(pathOut + "normal2." + str(i) + ".bed"):
        out.write(line.rstrip() + "\tnormal\t1\t1\n")
    out.close()
    os.system("cat " + pathOut + "normal3." + str(i) + ".bed " + pathOut + "dup." + str(i) + ".bed " + pathOut + "del." + str(i) + ".bed | " + bedtools_path + " sort -i stdin > " + pathOut + "total." + str(i) + ".bed")
    os.remove(pathOut + "normal3." + str(i) + ".bed")
    os.remove(pathOut + "normal2." + str(i) + ".bed")
    os.remove(pathOut + "normal." + str(i) + ".bed")
os.remove(pathOut + "del.bed")
os.remove(pathOut + "del2.bed")
os.remove(pathOut + "del3.bed")
os.remove(pathOut + "del4.bed")
os.remove(pathOut + "del5.bed")
os.remove(pathOut + "dup.bed")
os.remove(pathOut + "dup2.bed")
os.remove(pathOut + "dup3.bed")
os.remove(pathOut + "dup4.bed")
os.remove(pathOut + "dup5.bed")
os.remove(pathOut + "chrs.bed")

Removing overlaps, generating total file
Creating bedfiles for sample 1


In [ ]:
python3 dudeML.py simReads -fasta DiNV_CH01M.fa -cov 20 -d ${i}_sim -id ${i} -RL 100

python3 dudeML.py winStat -i${i}_sim/total.bam -o ${i}_sim/total_50.bed -w 50 -s 50

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
import os
"""
input is generated by genomeCoverageBed -d in the following format:
CHR POS COVERAGE
Following that, per chromosome, find the median coverage of covered bases.
Can find median for all chromosomes or a specified set of them, one chromosome ID per line.
"""
os.system(bedtools_path + " genomecov -d -ibam " + args.INPUT + " > dudeml_temp_covsperbase.bed")
print("Calculating median coverage")
test = pd.read_table("dudeml_temp_covsperbase.bed",header=None)
covs_median = {}
splits_median = {}
for line in open(args.CHROMOSOME):
    i = line.split()[0].rstrip()
    covs_median[i] = test[2][test[2] != 0][test[0] == i].median()
    print(i,covs_median[i])
if args.SUMMARY is not None:
    out = open(args.SUMMARY,"w")
    for i in covs_median:
        out.write(i + "\t" + str(covs_median[i]) + "\n")
    out.close()
if args.QUIET == False:
    print("Calculating relative median coverage per window")
chr_stats = []
count = 0
"function takes in a pandas dataframe column and outputs a dataframe containing the start and end of window, as well as window coverage median and standard deviation"
def rolling_with_step(chr,s, window, step):
    vert_idx_list = np.arange(1, s.size - window, step)
    hori_idx_list = np.arange(window)
    A, B = np.meshgrid(hori_idx_list, vert_idx_list)
    idx_array = A + B
    x_array = s.values[idx_array]
    idx = list(s.index[vert_idx_list + (int(window))])
    med = list(np.around(list(map(np.median, x_array)),4))
    intq = list(np.around(list(map(scipy.stats.iqr, x_array)),4))
    means = list(np.around(list(map(np.mean, x_array)),4))
    std = list(np.around(list(map(np.std, x_array)),4))
    return pd.DataFrame({"chr":chr,"start":vert_idx_list,"end":vert_idx_list + window,"med":med,"iqr":intq,"mean":means,"std":std})
out_df = pd.DataFrame(columns=["chr","start","end","med","iqr","mean","std"])
"""
For each chromosome, divide each base by the chromosome median (or total median).
Following that, finds the median and standard deviation for windows of a given size
"""
for i in covs_median:
    test_chrs = test[test[0] == i]
    test_chrs_3 = test_chrs[2]/covs_median[i]
    wins_step = rolling_with_step(i,test_chrs_3,args.WINDOW_SIZE-1,args.STEP_SIZE)
    if args.QUIET == False:
        print("Chromosome " + str(i) + " processed")
    out_df = pd.concat([out_df,wins_step])
out_df['chr']=out_df['chr'].astype(str)
out_df['start']=out_df['start'].astype(int)
out_df['end']=out_df['end'].astype(int)
out_df.to_csv(args.OUTPUT,sep="\t",index =False,columns=None,header=None)
os.remove("dudeml_temp_covsperbase.bed")